### Configuration

In [12]:
import os
import re
import numpy as np
import pandas as pd
import datetime

import mne
from scipy.stats import zscore
from tqdm import tqdm

from utils__helpers_macro import hilbert_powerphase, hilbert_envelope
from utils__helpers_epoch import epoch_sw, epoch_sw_2, epoch_spikes
import utils__config

In [13]:
os.chdir(utils__config.working_directory)
os.getcwd()

'Z:\\Layton\\Sleep_083023'

### Parameters

You will need to run this script once for **1s epochs** and once for **10s epochs.**

In [14]:
fif_path = 'Cache/Subject05/Jul12/S05_Jul12_256hz.fif'
bad_channel_path = 'Cache/Subject05/Jul12/S05_bad_channels.csv'
hypno_path = 'Cache/Subject05/Jul12/S05_Jul12_hypnogram.csv' 
sw_path = 'Cache/Subject05/Jul12/S05_SW.csv'
spike_path = 'Cache/Subject05/Jul12/S05_spikes.csv'
sw_out_path = 'Cache/Subject05/Jul12/S05_sw_epochs_1s.csv'
spike_out_path = 'Cache/Subject05/Jul12/S05_spike_epochs_1s.csv'
hypno_out_path = 'Cache/Subject05/Jul12/S05_hypno_epochs_1s.csv' 

In [15]:
epoch_length = 1 # bin width in seconds
sw_merge_threshold = 1 # how close SW's need to be for merging (in seconds)
sampling_freq = 256 # (s.f. used to detect slow waves)
hypno_sfreq = 256 # (s.f. used to make hypnogram)
tmin = 'none' # datetime.datetime(2022, 4, 28, 0, 0, 0, 0, tzinfo = datetime.timezone.utc)
tmax = 'none'
n_jobs = -2

### Load Data

In [16]:
raw = mne.io.read_raw_fif(fif_path, preload = True, verbose = None)

# Select only macroelectrodes
raw.pick_types(seeg = True, ecog = True)

# Remove rejected channels
bad_channels = pd.read_csv(bad_channel_path)
bad_channels = bad_channels[bad_channels['channel'].isin(raw.ch_names)]
raw.drop_channels(ch_names = bad_channels['channel'].astype('string'))

# Set the bin size; we set the stop interval
# to the total length of the recording
if tmin != 'none':

    last_bin = int((tmax - tmin).seconds / epoch_length) # will use this later
    bin_list = np.arange(0, (tmax - tmin).seconds + 1, epoch_length)

# Define tmin/tmax as the start/end of recording if not specified
else:
    
    tmin = raw.times[0]
    tmax = raw.times[-1]

    last_bin = int((tmax - tmin) / epoch_length) # will use this later
    bin_list = np.arange(0, (tmax - tmin) + 1, epoch_length)

Opening raw data file Cache/Subject05/Jul12/S05_Jul12_256hz.fif...


C:\Users\lal85\AppData\Local\Temp\2\ipykernel_40732\580800819.py:1: RuntimeWarning: This filename (Cache/Subject05/Jul12/S05_Jul12_256hz.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(fif_path, preload = True, verbose = None)


    Range : 0 ... 7249663 =      0.000 ... 28318.996 secs
Ready.
Opening raw data file Z:\Layton\Sleep_083023\Cache\Subject05\Jul12\S05_Jul12_256hz-1.fif...
    Range : 7249664 ... 9720319 =  28319.000 ... 37969.996 secs
Ready.
Reading 0 ... 9720319  =      0.000 ... 37969.996 secs...
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).


### Bin Hypnogram into Epochs

In [17]:
hypno = pd.read_csv(hypno_path, header = None)
hypno = hypno.reset_index()
hypno.columns = ['idx', 'stage']

# Bin the hypnogram into epochs based on sample number
hypno['epoch'] = pd.cut(hypno['idx'], bins = bin_list * hypno_sfreq, labels = False, include_lowest = True)

# Select the mode of the sleep stage within each epoch
hypno = hypno.groupby(['epoch'])['stage'].agg(pd.Series.mode).reset_index()

hypno.to_csv(hypno_out_path, index = False)

### Bin Delta Power into average per Epoch

In [18]:
# Extract Power and Phase
delta = raw.copy()
delta = hilbert_powerphase(data = delta, lower = 0.3, upper = 4, njobs = n_jobs)
delta = delta[['time', 'channel', 'power']]

# Calculate z-score of power 
delta['log_power'] = 10 * np.log10(delta['power'])
delta['zlog_power'] = delta.groupby(['channel'])['log_power'].transform(zscore)

# Extract Envelope
delta_env = raw.copy()
delta_env = hilbert_envelope(data = delta_env, lower = 0.3, upper = 4, njobs = n_jobs)
delta_env = delta_env[['time', 'channel', 'envelope']]

# Calculate z-score of envelope
delta_env['z_envelope'] = delta_env.groupby(['channel'])['envelope'].transform(zscore)

# Combine Power/Phase and Envelope
delta = delta.merge(delta_env, on = ['time', 'channel'])

# Bin the data with integer bin labels (pandas.cut 
# by default will create bins open on the left)
delta['epoch'] = pd.cut(delta['time'], bins = bin_list, labels = False)

# Average delta power by Epoch
delta = delta.groupby(['epoch', 'channel'])[['log_power', 'zlog_power', 'envelope', 'z_envelope']].mean().round(2).reset_index()

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.3 - 4 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.30
- Lower transition bandwidth: 0.30 Hz (-6 dB cutoff frequency: 0.15 Hz)
- Upper passband edge: 4.00 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 5.00 Hz)
- Filter length: 2817 samples (11.004 s)



[Parallel(n_jobs=-2)]: Using backend LokyBackend with 31 concurrent workers.
[Parallel(n_jobs=-2)]: Done  32 out of  51 | elapsed:    5.7s remaining:    3.3s
[Parallel(n_jobs=-2)]: Done  43 out of  51 | elapsed:    6.2s remaining:    1.1s
[Parallel(n_jobs=-2)]: Done  51 out of  51 | elapsed:    6.5s finished


Converting "channel" to "category"...
Converting "ch_type" to "category"...


C:\Users\lal85\AppData\Local\Temp\2\ipykernel_40732\2950059664.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  delta['zlog_power'] = delta.groupby(['channel'])['log_power'].transform(zscore)


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.3 - 4 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.30
- Lower transition bandwidth: 0.30 Hz (-6 dB cutoff frequency: 0.15 Hz)
- Upper passband edge: 4.00 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 5.00 Hz)
- Filter length: 2817 samples (11.004 s)



[Parallel(n_jobs=-2)]: Using backend LokyBackend with 31 concurrent workers.
[Parallel(n_jobs=-2)]: Done  32 out of  51 | elapsed:    4.0s remaining:    2.3s
[Parallel(n_jobs=-2)]: Done  43 out of  51 | elapsed:    5.3s remaining:    0.9s
[Parallel(n_jobs=-2)]: Done  51 out of  51 | elapsed:    5.8s finished


Converting "channel" to "category"...
Converting "ch_type" to "category"...


C:\Users\lal85\AppData\Local\Temp\2\ipykernel_40732\2950059664.py:16: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  delta_env['z_envelope'] = delta_env.groupby(['channel'])['envelope'].transform(zscore)
C:\Users\lal85\AppData\Local\Temp\2\ipykernel_40732\2950059664.py:26: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  delta = delta.groupby(['epoch', 'channel'])[['log_power', 'zlog_power', 'envelope', 'z_envelope']].mean().round(2).reset_index()


### Bin Beta Power into average per Epoch

In [19]:
# Extract Power and Phase
beta = raw.copy()
beta = hilbert_powerphase(data = beta, lower = 15, upper = 30, njobs = n_jobs)
beta = beta[['time', 'channel', 'power']]

# Calculate z-score of power and envelope
beta['beta_log_power'] = 10 * np.log10(beta['power'])
beta['beta_zlog_power'] = beta.groupby(['channel'])['beta_log_power'].transform(zscore)

# Bin the data with integer bin labels (pandas.cut 
# by default will create bins open on the left)
beta['epoch'] = pd.cut(beta['time'], bins = bin_list, labels = False)

# Average beta power by Epoch
beta = beta.groupby(['epoch', 'channel'])[['beta_log_power', 'beta_zlog_power']].mean().round(2).reset_index()

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 15 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 15.00
- Lower transition bandwidth: 3.75 Hz (-6 dB cutoff frequency: 13.12 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 227 samples (0.887 s)



[Parallel(n_jobs=-2)]: Using backend LokyBackend with 31 concurrent workers.
[Parallel(n_jobs=-2)]: Done  32 out of  51 | elapsed:    9.1s remaining:    5.3s
[Parallel(n_jobs=-2)]: Done  43 out of  51 | elapsed:    9.7s remaining:    1.7s
[Parallel(n_jobs=-2)]: Done  51 out of  51 | elapsed:   10.3s finished


Converting "channel" to "category"...
Converting "ch_type" to "category"...


C:\Users\lal85\AppData\Local\Temp\2\ipykernel_40732\2293257740.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  beta['beta_zlog_power'] = beta.groupby(['channel'])['beta_log_power'].transform(zscore)
C:\Users\lal85\AppData\Local\Temp\2\ipykernel_40732\2293257740.py:15: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  beta = beta.groupby(['epoch', 'channel'])[['beta_log_power', 'beta_zlog_power']].mean().round(2).reset_index()


### Bin SW's into duration per Epoch

In [20]:
sw = epoch_sw_2(sw_path = sw_path, 
                tmin = tmin, 
                tmax = tmax, 
                merge_threshold = sw_merge_threshold, 
                sampling_freq = sampling_freq, 
                bin_list = bin_list)

  0%|          | 0/27 [00:00<?, ?it/s]z:\Layton\Sleep_083023\Source\utils__helpers_epoch.py:165: FutureWarning: The behavior of obj[i:j] with a float-dtype index is deprecated. In a future version, this will be treated as positional instead of label-based. For label-based slicing, use obj.loc[i:j] instead
  bool_series[sw_row.start:sw_row.stop] = True
  4%|▎         | 1/27 [00:00<00:16,  1.55it/s]z:\Layton\Sleep_083023\Source\utils__helpers_epoch.py:165: FutureWarning: The behavior of obj[i:j] with a float-dtype index is deprecated. In a future version, this will be treated as positional instead of label-based. For label-based slicing, use obj.loc[i:j] instead
  bool_series[sw_row.start:sw_row.stop] = True
  7%|▋         | 2/27 [00:01<00:15,  1.65it/s]z:\Layton\Sleep_083023\Source\utils__helpers_epoch.py:165: FutureWarning: The behavior of obj[i:j] with a float-dtype index is deprecated. In a future version, this will be treated as positional instead of label-based. For label-based sli

In [21]:
# Merge the Delta Power, Beta Power, and Slow Wave Duration data
sw_delta = delta.merge(sw, on = ['epoch', 'channel'])
sw_delta = sw_delta.merge(beta, on = ['epoch', 'channel'])

sw_delta.to_csv(sw_out_path, index = False)

### Bin spikes into FR per Epoch

In [22]:
spikes = epoch_spikes(spike_path = spike_path, 
                      bin_width = epoch_length, 
                      bin_list = bin_list, 
                      last_bin = last_bin)

spikes.to_csv(spike_out_path, index = False)